In [74]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [75]:
train_data_dir='/home/aryan/Train'
val_data_dir='/home/aryan/Val'

In [76]:
img_rows,img_cols=224,224
model=VGG16(weights ='imagenet',
       include_top=False,
       input_shape=(img_rows,img_cols,3))

In [77]:
for layer in model.layers:
  layer.trainable = False

In [78]:
folders = glob('/home/aryan/Train/*')

In [79]:
# our layers - you can add more if you want
x = Flatten()(model.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [80]:
model=Model(inputs=model.input,outputs=prediction)

In [81]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [82]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [83]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(val_data_dir,
                                            target_size = (224, 224),
                                            batch_size = 15,
                                            class_mode = 'categorical')

Found 304 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [84]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
10/10 [==============================] - 108s 11s/step - loss: 1.8837 - accuracy: 0.3300 - val_loss: 0.6465 - val_accuracy: 0.6500
Epoch 2/5
10/10 [==============================] - 109s 11s/step - loss: 0.5914 - accuracy: 0.7661 - val_loss: 0.3318 - val_accuracy: 0.7800
Epoch 3/5
10/10 [==============================] - 108s 11s/step - loss: 0.1599 - accuracy: 0.9829 - val_loss: 0.0514 - val_accuracy: 1.0000
Epoch 4/5
10/10 [==============================] - 108s 11s/step - loss: 0.0994 - accuracy: 0.9845 - val_loss: 0.0269 - val_accuracy: 1.0000
Epoch 5/5
10/10 [==============================] - 108s 11s/step - loss: 0.0554 - accuracy: 0.9972 - val_loss: 0.0392 - val_accuracy: 1.0000


In [70]:
model.save('faceDetector.h5')

In [71]:
from keras.models import load_model
model=load_model('faceDetector.h5')

In [72]:
import os
import cv2
from keras.preprocessing import image
from PIL import Image

In [73]:
for i in os.listdir('/home/aryan/Val'):
    path='/home/aryan/Val/'
    print(path+i)
    face=cv2.imread(path+i)
    print(type(face))
    if type(face) is np.ndarray:
        face=cv2.resize(face,(224,224))
        #face=cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        plt.imshow(face)
        plt.show()
        im=Image.fromarray(face,'RGB')
        img_array=np.array(im)
        img_array=np.expand_dims(img_array,axis=0)
        pred=model.predict(img_array)
        print(pred)
        classes=np.argmax(pred,axis=1)
        print(classes)
        

/home/aryan/Val/Kittu
<class 'NoneType'>
/home/aryan/Val/Aryan
<class 'NoneType'>
/home/aryan/Val/Mukhil
<class 'NoneType'>
/home/aryan/Val/Alvin
<class 'NoneType'>
